In [1]:
# 4.2 Volatility Spillover Measures Using GVD on Realized Covariances (vech)

import pandas as pd
import numpy as np
from statsmodels.tsa.api import VAR
from joblib import Parallel, delayed
from tqdm import tqdm
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- UI for dataset selection ---
option_selector = widgets.Dropdown(
    options=['europe', 'australia'],
    value='europe',
    description='Dataset:',
)
button = widgets.Button(description="Load Dataset", button_style='success', icon='check')

# --- Dataset loader ---
def on_button_click(b):
    global selected_option, pit_vech
    clear_output(wait=True)
    display(option_selector, button)
    selected_option = option_selector.value
    file_path = f"parquet_files/pit_transformed_vech_{selected_option}.parquet"
    pit_vech = pd.read_parquet(file_path)
    pit_vech.index = pd.to_datetime(pit_vech.index)
    print(f"✅ Loaded: {file_path}")
    display(pit_vech.head())

button.on_click(on_button_click)
display(option_selector, button)


Dropdown(description='Dataset:', index=1, options=('europe', 'australia'), value='australia')

Button(button_style='success', description='Load Dataset', icon='check', style=ButtonStyle())

,nsw,qld-nsw,qld,sa-nsw,sa-qld,sa,tas-nsw,tas-qld,tas-sa,tas,vic-nsw,vic-qld,vic-sa,vic-tas,vic
2009-07-01,-0.699212,-0.895466,-1.271926,-0.366649,-0.786238,-0.097437,-0.118865,-0.045462,-0.062529,-0.011362,-0.630993,-0.582337,-0.674936,-0.081039,-0.901846
2009-07-02,0.282052,0.749995,1.040695,0.096010,-1.273522,0.046172,0.167654,0.316959,0.379591,-0.024856,0.382645,-0.273933,0.376540,0.512218,0.442966
2009-07-03,-0.661619,-0.362096,-0.563926,-0.466590,-0.122441,-0.589925,-0.508174,0.031960,-0.089591,-0.683880,-0.229950,-0.092444,-0.199430,-0.108145,-0.319200
2009-07-04,-0.720222,-0.597548,-0.947581,-0.626666,-0.304291,-0.711049,-0.187131,-0.073205,-0.172697,-0.065375,-0.413391,-0.276146,-0.375016,-0.208130,-0.551482
2009-07-05,-0.212486,0.012072,-0.117435,-0.045462,0.282791,-0.153986,0.094583,0.596699,0.413391,-0.218300,0.137480,0.296120,0.167654,0.412618,0.083889


In [3]:

# --- Fit VAR model ---
p = 30
model = VAR(pit_vech)
results = model.fit(p)


C:\Users\frolo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


In [4]:

# --- Compute MA(\infty) coefficients ---
H = 1
K = pit_vech.shape[1]
I = np.eye(K)
A = [I]
Phi = results.coefs

for h in range(1, H):
    A_h = np.zeros((K, K))
    for j in range(1, min(h, p)+1):
        A_h += Phi[j-1] @ A[h-j]
    A.append(A_h)

# --- GVD computation ---
Sigma = results.sigma_u
sigma_diag_inv = np.diag(1 / np.diag(Sigma))
theta_g = np.zeros((K, K))

for i in range(K):
    e_i = np.zeros(K)
    e_i[i] = 1
    denom = sum([e_i @ A[h] @ Sigma @ A[h].T @ e_i for h in range(H)])
    for j in range(K):
        e_j = np.zeros(K)
        e_j[j] = 1
        numer = sum([(e_i @ A[h] @ Sigma @ e_j)**2 for h in range(H)])
        theta_g[i, j] = sigma_diag_inv[j, j] * numer / denom

# --- Normalize and compute TSI ---
theta_g_normalized = theta_g / theta_g.sum(axis=1, keepdims=True)
TSI = 100 * (np.sum(theta_g_normalized) - np.trace(theta_g_normalized)) / K

# --- Spillover matrix ---
spillover_matrix = theta_g_normalized * 100

# --- Directional measures ---
directional_to = spillover_matrix.sum(axis=0) - np.diag(spillover_matrix)
directional_from = spillover_matrix.sum(axis=1) - np.diag(spillover_matrix)
net_directional = directional_to - directional_from

spillover_table = pd.DataFrame(spillover_matrix,
                               index=pit_vech.columns,
                               columns=pit_vech.columns)
spillover_table["Directional FROM others"] = directional_from
spillover_table.loc["Directional TO others"] = list(directional_to) + [directional_to.sum()]
spillover_table.loc["NET Directional"] = list(net_directional) + [np.nan]

# --- Save ---
spillover_table.to_parquet(f"parquet_files/volatility_spillovers_VAR_fullcov_{selected_option}.parquet")
print(f"Total Spillover Index (TSI): {TSI:.2f}%")
spillover_table


Total Spillover Index (TSI): 65.52%


,nsw,qld-nsw,qld,sa-nsw,sa-qld,sa,tas-nsw,tas-qld,tas-sa,tas,vic-nsw,vic-qld,vic-sa,vic-tas,vic,Directional FROM others
nsw,27.718905,6.939005,11.266904,10.896348,1.261236,12.164804,3.096098,0.789588,1.075966,5.942891,3.690055,1.586689,2.715365,2.380984,8.475162,72.281095
qld-nsw,9.316452,37.215974,23.562666,1.039588,7.781912,0.582907,0.622667,3.026921,1.597319,0.364312,3.448274,5.227350,2.747109,0.981360,2.485190,62.784026
qld,15.350522,23.910557,37.765448,1.755262,2.340938,1.492655,0.591882,1.247225,0.910097,0.794834,3.035617,3.004036,2.081591,1.141080,4.578256,62.234552
sa-nsw,11.362478,0.807421,1.343431,28.904681,3.192600,21.254211,1.418675,1.621352,1.373180,1.840587,6.868712,2.878898,3.349092,3.879444,9.905237,71.095319
sa-qld,1.927567,8.858214,2.625938,4.679134,42.363243,0.988549,1.747086,5.881262,1.377150,0.286418,5.522728,17.349854,3.169704,0.564935,2.658218,57.636757
sa,14.009873,0.500005,1.261740,23.473722,0.744928,31.923107,0.866821,0.413595,0.774849,2.395381,3.775166,0.825671,3.047877,2.933857,13.053407,68.076893
tas-nsw,3.803585,0.569746,0.533697,1.671357,1.404364,0.924653,34.052929,4.548012,9.793139,19.530532,6.338148,1.933971,4.996577,6.559728,3.339563,65.947071
tas-qld,1.300251,3.712567,1.507483,2.560423,6.337003,0.591389,6.096346,45.645979,6.598216,1.186106,3.847442,9.011867,1.166736,8.929330,1.508862,54.354021
tas-sa,1.368518,1.513179,0.849613,1.674894,1.146093,0.855736,10.139006,5.096265,35.255588,2.749357,8.679144,1.119976,9.003935,16.384744,4.163953,64.744412
tas,8.612824,0.393250,0.845484,2.558068,0.271604,3.014348,23.040054,1.043866,3.132760,40.172041,3.281295,0.611912,2.558851,4.460166,6.003478,59.827959
